In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image


In [8]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

# Paths to dataset directories
frames_dirs = {
    'Training': r'P:\frames_preprocessing\Train',
    'Validation': r'P:\frames_preprocessing\Validation',
    'Test': r'P:\frames_preprocessing\Test',
}

features_dirs = {
    'Training': r'P:\feature_extraction\Training',
    'Validation': r'P:\feature_extraction\Validation',
    'Test': r'P:\feature_extraction\Test',
}

# Ensure output directories exist
for dir_path in features_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# Load ResNet50 model (pretrained on ImageNet)
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")
model = Model(inputs=base_model.input, outputs=base_model.output)  # Output shape: (2048,)

# Function to extract features in batches
def extract_features(img_paths, batch_size=16):
    batch_images = []
    features = []

    for img_path in img_paths:
        try:
            img = load_img(img_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = preprocess_input(img_array)
            batch_images.append(img_array)
        except Exception as e:
            print(f"⚠️ Error loading {img_path}: {e}")
            continue

        if len(batch_images) == batch_size:
            batch_images = np.array(batch_images)
            batch_features = model.predict(batch_images, batch_size=batch_size, verbose=0)
            features.append(batch_features)
            batch_images = []  # Reset batch

    if batch_images:  # Process remaining images
        batch_images = np.array(batch_images)
        batch_features = model.predict(batch_images, batch_size=batch_size, verbose=0)
        features.append(batch_features)

    return np.vstack(features) if features else None  # Return combined feature array

# Function to extract features for a dataset
def extract_data_features(data_type, batch_size=16):
    print(f"🚀 Starting feature extraction for {data_type}...")
    frames_dir = frames_dirs[data_type]
    features_dir = features_dirs[data_type]

    for person in os.listdir(frames_dir):
        person_path = os.path.join(frames_dir, person)
        if not os.path.isdir(person_path):
            continue

        person_features_dir = os.path.join(features_dir, person)
        os.makedirs(person_features_dir, exist_ok=True)

        for video in os.listdir(person_path):
            video_path = os.path.join(person_path, video)
            if not os.path.isdir(video_path):
                continue

            frame_files = sorted([f for f in os.listdir(video_path) if f.endswith((".jpg", ".png"))])
            frame_paths = [os.path.join(video_path, f) for f in frame_files]

            print(f"🔹 Processing {data_type} - {person}/{video} ({len(frame_paths)} frames)")

            # Ensure exactly 50 frames per video
            if len(frame_paths) != 50:
                print(f"⚠️ Warning: {data_type}/{person}/{video} has {len(frame_paths)} frames (expected 50). Skipping!")
                continue

            features = extract_features(frame_paths, batch_size=batch_size)

            if features is not None:
                np.save(os.path.join(person_features_dir, f"{video}.npy"), features)
                print(f"✅ Saved: {data_type}/{person}/{video}.npy")

    print(f"✅ Feature extraction for {data_type} completed!")

# Set batch size (adjust based on GPU memory)
BATCH_SIZE = 16  # Adjust for performance

# Run feature extraction for all datasets
for dataset in ["Training", "Validation", "Test"]:
    extract_data_features(dataset, batch_size=BATCH_SIZE)


🚀 Starting feature extraction for Training...
🔹 Processing Training - 110001/1100011002 (50 frames)
✅ Saved: Training/110001/1100011002.npy
🔹 Processing Training - 110001/1100011003 (50 frames)
✅ Saved: Training/110001/1100011003.npy
🔹 Processing Training - 110001/1100011004 (50 frames)
✅ Saved: Training/110001/1100011004.npy
🔹 Processing Training - 110001/1100011005 (50 frames)
✅ Saved: Training/110001/1100011005.npy
🔹 Processing Training - 110001/1100011006 (50 frames)
✅ Saved: Training/110001/1100011006.npy
🔹 Processing Training - 110001/1100011007 (50 frames)
✅ Saved: Training/110001/1100011007.npy
🔹 Processing Training - 110001/1100011008 (50 frames)
✅ Saved: Training/110001/1100011008.npy
🔹 Processing Training - 110001/1100011009 (50 frames)
✅ Saved: Training/110001/1100011009.npy
🔹 Processing Training - 110001/1100011010 (50 frames)
✅ Saved: Training/110001/1100011010.npy
🔹 Processing Training - 110001/1100011011 (50 frames)
✅ Saved: Training/110001/1100011011.npy
🔹 Processing T

KeyboardInterrupt: 